I have chosen a place called Chennai, India.In the below code, I tried to find all the last words and see if it is right.When 
I saw the output most of the last words seemed to be right. Very few words had wrong last words. So, what I did was I removed
all the wrong last words and made it into a list called "wrong_words" which you can see in the next cell. Reason for doing this 
rather than collecting all the right last words and putting that in a list as we did in the classroom quiz is that there are way 
too many right words than the wrong words. And it becomes difficult to put that many right words in the list. 

In [1]:
#Please note that in this cell, I have analysed the actual osm file downloaded from the zen map and the sample of it is analysed 
# in the third cell
    
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
# My file name
osm_file="chennai_india.osm"
#Few names which I found right
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons","Road","Salai","Luz","Velacherry Village"," Nungambakkam"]
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


#checking the last word if it is in the list and appending it if it is not.
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types.append(street_name)
            
   
#Checking if the "k" is addr:street
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


#list where all the last words not found in the "expected" list is appended
street_types=[]
for event, elem in ET.iterparse(osm_file, events=("start",)):
    #print elem.tag

    if elem.tag == "node" or elem.tag == "way":
        for tag in elem.iter("tag"):
            if is_street_name(tag):
                audit_street_type(street_types, tag.attrib['v'])

#This set is to find the unique last word  
unique=set(street_types)

for ways in unique:
    print ways

P.A. Koil 2nd street
opp. Hotel Saravana Bhavan,Anna Main Road, K K Nagar
Annanagar Western Extension, J Block, Welcome Colony, Thirumangalam
92nd St, 18th avenue, Ashok Nagar
2nd cross street, bharathi nagar, ambattur
Old Slaughterhouse Rd
sriram nagar
Gandhi Nagar, MGR Nagar, Venkateswara Nagar, Velachery, Chennai
6th cross ,29th main, BTM Layout 2nd Stage
5th Cross Street, Govindsamay Nagar, Madipakkam
Gandhi road

VANNIPAKKAM TO JAYARAMAPURAM VILLAGE ROAD
thiruporur,kanchipuram
Sivagnanam Street, T Nagar
J.N.ROAD
4th Avenue,Shanthi Colony,Anna Nagar,Chennai - 600040,Tamil Nadu India
Samraj Nagar
2nd Ave
Vallaba Agraharam Street, Tiruvatteeswaranpet
Ambattur Estate Rd
Spurtank Road, Chetpet
Defence Officers Colony Nandambakkam Extension Guindy
Guhan Street, Puliyur, Kodambakkam
Jayachandran Nagar
Nagapattinam-Chennai Highway, NGR Colony, Gotha Medu Housing Board, Saidapet, NGR Colony, Gotha Medu Housing Board, West Saidapet
New Street, West Mambalam
2nd street
Haddows 2nd Street, Nu

In the below code, I am checking if the postal codes is in the range between 600001 and 600113. i found that 7 values are out 
range

In [8]:
import xml.etree.cElementTree as ET
file_name="chennai_india.osm"
context=ET.iterparse(file_name)
list=[]
for event,element in context:
    if element.tag=='node':
        for y in element.iter("tag"): 
            if y.attrib['k']=='postal_code' or y.attrib['k']=='addr:postcode':
                if int(y.attrib['v'].strip())<600001 or int(y.attrib['v'].strip())>600113:
                    list.append(y.attrib['v'])
print len(list)

7


In [9]:
# In this cell, I have written the code to clean the "addr:street" value and "postalcode" of the area.
# In this case, I have used the sample osm file.
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint
import codecs
import csv
import schema
import cerberus





OSMFILE = "sample.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')

SCHEMA = schema.schema

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

# As stated in the above cell, all the wrong last words is included in the list. If you see few last words have postal code,
#few have punctuations at the end and few have words are caplital letters like "AVENUE","ROAD"


wrong_words = ["Theatre)","street)","Rd","STREET","St.","ROAD","T.Nagar,","J.N.ROAD","III",
            "Nagar,","Post,","SALAI","nagar","street","HO","AVENUE","Adyar,","Road)","Guduvanchery,","600040","600042","ST",
"strret","Extn","II","NAGAR","road","NADU,INDIA","600050","West,5","LANE","Street,","Strret","road)","PERUNGUDI",
"St","EXTN","Pudur)",
"Goparasanallur,","Ave","600","SH49","Road,kodambakkam","600037","Navallur,","Broadway)","East,","600101","Valasaravakkam,",
"Kancheepuram","Velachery,","Road,Periyapet,","Nagar,,","st","Extn."]

#In this dictionary, I have included the wrong words as keys and it's correct words as values.
#But I have not included all the wrong words in this dictionary as I have cleaned few last words in the update_name function 
#directly 
mapping={"Rd":"Road",
        "STREET":"Street",
        "St.":"Street",
        "ROAD":"Road",
        "strret":"Street",
        "Extn":"Extension",
        "NAGAR":"Nagar",
        "SALAI":"Salai",
        "nagar":"Nagar",
        "street":"Street",
        "AVENUE":"Avenue",
        "ST":"Street",
        "road":"Road",
        "NADU INDIA":"Nadu India",
        "LANE":"Lane",
        "PERUNGUDI":"Perungudi",
        "Ave":"Avenue",
        "Kancheepuram":"Kanchipuram",
        "st":"Street",
        "Extn.":"Extension"}
# In the below code,

# Finding keys in "mapping" dictionary which is useful in the "update_name" function.
keys= mapping.keys()
#In the below function, I am correcting the last words in the "addr:street" and returning it.
def update_name(name, mapping):
       
       
#I am checking if the last word is in the dictionary "mapping" keys,if so I am correcting the last word and returning the
#whole word. If the last word is not in the "wrong_word" list it will move on to else.
        m = street_type_re.search(name)
        if m:
            street_type=m.group()
    
            if street_type in keys:
                value=mapping[street_type]
                y=name.find(street_type)
                x=name[:y]
                z=x +value
                return z
#This is for words which have ")", "," and numbers at the end of the last word of the street as you can see in list "wrong_words" 

# and those words has been corrected
            else:
                try: 
            #If the street_type had numerals, I removed it.
                    type(int(street_type))
                    position=name.find(street_type)
                    remove_numbers=name[:position]
                    return remove_numbers
            #If the street_type had strings other than what it is in mapping and numbers
            #If strings had "," and ")" in the last word, that is why used replace
                except ValueError:
                    x = name.replace(","," ").replace(")"," ")
                    return x
                 
# I found out that few postal codes of the city is out of range that is it has to be between 600001 and 600113.
# Postal_code other than that is not correct so, returned None for such values.

def update_postalcode(code):
    
#Checking if the postalcode is out range and returning None

    
    try:
        if int(code)<600001 or int(code)<600113:
            code1=None
            return code1
        else:
            code1=code
            return code1
    except ValueError:
        return None
    
#Returning the original value if the postal code is in range.

   
        
#In this function, I have called the above functions to clean the address and postal code of the city and converted the xml to
#dictionary

def shape_element(element, default_tag_type='regular'):
   
    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements
    node_tag={}
  
#Check if the tag is "node

    if element.tag == 'node':
        
      
#looping over the attributes in the element and putting the values in the dictionary
        for i in element.attrib:
            for x in NODE_FIELDS:
                node_attribs[x]=element.attrib[x]
# Checking if the element has the secondary tag "tag"
        
        for y in element:
            if y.tag=='tag':
       
            
#Check if the tag "k" value is problematic, if so move on to next in the loop.       

                if PROBLEMCHARS.search(y.attrib["k"]):
                    continue
            
#Check if the "k" value have colon in it, if so equate the characters before the first colon to "type" key in the dictionary and
#equate the rest of the characters after the first colon to "key" key in the dictionary
# And equate the element id to the "id" key of the dictionary

                elif LOWER_COLON.match(y.attrib["k"]):
                    node_tag["type"] = y.attrib["k"].split(":", 1)[0]
                    node_tag["key"] = y.attrib["k"].split(":", 1)[1]
                    node_tag["id"] = element.attrib["id"]

#Checking if the"k"value is "addr:street", if so calling the upadte_name function and equating it to the "value" key in the 
#dictionary
                    if y.attrib["k"] == 'addr:street':
                        node_tag["value"] = update_name(y.attrib["v"], mapping)
            
#Checking if the "k" is "addr:postcode" , if so updating the postal code using the updat_postalcode function.

                    elif y.attrib['k']=='addr:postcode':
                        if  update_postalcode(y.attrib['v']):
                             node_tag["value"]=update_postalcode(y.attrib["v"])
                        else:
                            continue
                            
#Below code is for the "k" not equal to "addr:street"
                    else:
                        node_tag["value"] = y.attrib["v"]
                
# Below code is to check if the "k" is 'post_code' and changing the value of the postal code using the
#update_postalcode function 

                elif y.attrib['k']=='post_code':
                    if update_postalcode(y.attrib['v']):
                        node_tag["value"]=update_postalcode(y.attrib["v"])
                        node_tag["type"]='regular'
                        node_tag["key"]=y.attrib["k"]
                        node_tag["id"]=element.attrib["id"]
                    else:
                        continue
                        
#Below code is for the rest of the "k" if it doesn't satisfy the above conditions.

                else:
                    node_tag["type"]='regular'
                    node_tag["key"]=y.attrib["k"]
                    node_tag["id"]=element.attrib["id"]
                    node_tag["value"]=y.attrib["v"]
           
                tags.append(node_tag)
            
                        
                                    
                                
          
          
            

        return {'node': node_attribs, 'node_tags': tags}
# The below code is as same as the above code in this function only difference is that it is for thr "way" tag.
    elif element.tag == 'way':
        
        for j in element.attrib:
                
            for x in WAY_FIELDS:
                way_attribs[x]=element.attrib[x]
           
            i=0
        for q in element.iter("nd"):
            
           
            way_nodes.append({'id':element.attrib["id"],'node_id': q.attrib["ref"],'position':i})
            i=i+1
     

    
        for x in element:
            if x.tag=='tag':
            
            
                if PROBLEMCHARS.search(x.attrib["k"]):
                    continue
                elif LOWER_COLON.match(x.attrib["k"]):
                    node_tag["type"] = x.attrib["k"].split(":", 1)[0]
                    node_tag["key"] = x.attrib["k"].split(":", 1)[1]
                    node_tag["id"] = element.attrib["id"]

                
                    if x.attrib["k"] == 'addr:street':
                        node_tag["value"] = update_name(x.attrib["v"], mapping)
               
                    elif x.attrib['k']=='addr:postcode':
                        if update_postalcode(x.attrib['v']):
                             node_tag["value"]=update_postalcode(x.attrib["v"])
                        else:
                            continue
                            
                    else:
                        node_tag["value"] = x.attrib["v"]
                elif x.attrib['k']=='post_code':
                    if update_postalcode(x.attrib['v']):
                        node_tag["value"]=update_postalcode(x.attrib["v"])
                        node_tag["type"]='regular'
                        node_tag["key"]=x.attrib["k"]
                        node_tag["id"]=element.attrib["id"]
                    else:
                        continue
                else:
                    node_tag["type"]='regular'
                    node_tag["key"]=x.attrib["k"]
                    node_tag["id"]=element.attrib["id"]
                    node_tag["value"]=x.attrib["v"]
           
                tags.append(node_tag)
            
                
      
 
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem           
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(OSMFILE, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(OSMFILE, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSMFILE, validate=True)
